In [1]:
import matplotlib.pyplot as plt
from numpy import arange, zeros
from pathlib import Path
from os import listdir
import ipywidgets as widgets
from IPython.display import display
from dependencies import auto_correlation, cross_correlation
%matplotlib widget


def on_change_select_first_signal(change):
    with out:
        texts0 = [[], []] # X(sequence) and Y(ecg signal) value
        with open(select_first_signal.get_interact_value(), "r") as file:
            text_file = file.readlines()
        texts0[0] = arange(len(text_file))
        try:
            for text in text_file: 
                texts0[1].append(float(text.replace("\n", "")))
        except ValueError:
            print("Please select single column data file.")
            return
        fig0_ax0.clear()
        fig0_ax0.plot(texts0[0], texts0[1], label="first signal")
        fig0.legend()
        first_signal_length.value = f"{len(texts0[0])}"
        
        
def on_change_select_second_signal(change):
    with out:
        texts0 = [[], []] # X(sequence) and Y(ecg signal) value
        with open(select_second_signal.get_interact_value(), "r") as file:
            text_file = file.readlines()
        texts0[0] = arange(len(text_file))
        try:
            for text in text_file: 
                texts0[1].append(float(text.replace("\n", "")))
        except ValueError:
            print("Please select single column data file.")
            return
        fig0_ax1.clear()
        fig0_ax1.plot(texts0[0], texts0[1], color="green", label="second signal")
        fig0.legend()
        second_signal_length.value = f"{len(texts0[0])}"
        
        
def on_change_time_lag(change):
    with out:
        texts2 = [[], []] # X(sequence) and Y(ecg signal) time lagged value
        texts2[0] = texts1[0]
        texts2[1] = zeros(len(texts1[1]))
        time_lag = select_time_lag.get_interact_value()
        if time_lag > 0:
            for x in range(time_lag, len(texts2[1])):
                texts2[1][x] = texts1[1][x - time_lag]
        elif time_lag < 0:
            for x in range(abs(time_lag), len(texts2[1])):
                texts2[1][x - abs(time_lag)] = texts1[1][x]
        else:
            texts2[1] = texts1[1]
        fig2_ax1.clear()
        fig3_ax0.clear()
        fig2_ax1.plot(texts2[0], texts2[1], color="green")
        fig3_ax0.plot([x - (len(texts0[0]))//2 for x in texts0[0]], cross_correlation("normal", texts0[1], texts2[1]), label="Correlated Signal")

In [2]:
print("Please select the same data length for first and second signal")
files = listdir(Path().absolute())

select_first_signal = widgets.Select(
    options=files,
    description='Select First File:',
    disabled=False
)

first_signal_length = widgets.Text(value='0', disabled=True)

select_second_signal = widgets.Select(
    options=files,
    description='Select Second File:',
    disabled=False
)

second_signal_length = widgets.Text(value="0", disabled=True)
out = widgets.Output()
vertical_box_1 = widgets.VBox([select_first_signal, first_signal_length])
vertical_box_2 = widgets.VBox([select_second_signal, second_signal_length])
horizontal_box = widgets.HBox([vertical_box_1, vertical_box_2])

select_first_signal.observe(on_change_select_first_signal)
select_second_signal.observe(on_change_select_second_signal)

display(horizontal_box)

fig0 = plt.figure()
fig0_ax0 = fig0.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude")
fig0_ax1 = fig0_ax0.twinx()
fig0_ax1.set_ylabel(ylabel="amplitude")
plt.show()

Please select the same data length for first and second signal


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
texts0 = [[], []] # X(sequence) and Y(ecg signal) value
texts1 = [[], []] # X(sequence) and Y(ecg signal) value

with open(select_first_signal.get_interact_value(), "r") as file:
    text_file = file.readlines()
    
texts0[0] = arange(len(text_file))

for text in text_file: 
    texts0[1].append(float(text.replace("\n", "")))

with open(select_second_signal.get_interact_value(), "r") as file:
    text_file = file.readlines()
    
texts1[0] = arange(len(text_file))

for text in text_file: 
    texts1[1].append(float(text.replace("\n", "")))


In [ ]:
fig1 = plt.figure()
fig1_ax0 = fig1.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Auto Correlation")
fig1_ax0.plot(texts0[0], [abs(x) for x in auto_correlation("normal", texts0[1])], label="Phase Shift")
fig1_ax0.legend()
fig1_ax1 = fig1_ax0.twinx()
fig1_ax1.set_ylabel(ylabel="amplitude")
fig1_ax1.plot(texts0[0], [abs(x) for x in auto_correlation("inversefourier", texts0[1])], label="Inverse Fourier", color="green")
fig1_ax1.legend()
plt.show()

In [5]:
select_time_lag = widgets.IntSlider(
    value=0,
    min=-(len(texts1[0]//2)),
    max=len(texts1[0]//2),
    step=1,
    description='Time Lag:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

out = widgets.Output()

display(select_time_lag, out)
select_time_lag.observe(on_change_time_lag)

fig2 = plt.figure()
fig2_ax0 = fig2.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude")
fig2_ax0.plot(texts0[0], texts0[1])
fig2_ax1 = fig2_ax0.twinx()
fig2_ax1.plot(texts1[0], texts1[1], color="green")
fig2_ax1.set_ylabel(ylabel="amplitude")

fig3 = plt.figure()
fig3_ax0 = fig3.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude")
fig3_ax0.plot([x - (len(texts0[0]))//2 for x in texts0[0]], cross_correlation("normal", texts0[1], texts1[1]), label="Correlated Signal")
fig3_ax0.legend()
plt.show()

IntSlider(value=0, continuous_update=False, description='Time Lag:', max=512, min=-512)

Output()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …